In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.base import TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv("data.csv") 
df['Loan_Status'] = df['Loan_Status'].replace({'Y':1, 'N':0})
# create X,y
y = df.pop('Loan_Status')
X = df.drop('Loan_ID',axis=1)

In [3]:
class DataframeFunctionTransformer:
    def __init__(self, func):
        self.func = func
    def transform(self, input_df, **transform_params):
        return self.func(input_df)
    def fit(self, X, y=None, **fit_params):
        return self
def create_total_income_feature(input_df):
    input_df['total_income'] = input_df['ApplicantIncome'] + input_df['CoapplicantIncome']
    return input_df
def to_dataframe(array):
    columns= ['Gender','Dependents','Married','Self_Employed','LoanAmount',
               'Loan_Amount_Term','Credit_History','Education','ApplicantIncome',
               'CoapplicantIncome','Property_Area','total_income']
    return pd.DataFrame(array, columns = columns)

In [4]:
fillna_trans = ColumnTransformer([
     ('fillna_mode', SimpleImputer(strategy="most_frequent"), ['Gender','Dependents']),
     ('fillna_No', SimpleImputer(strategy='constant', fill_value='No'), ['Married', 'Self_Employed']),
     ('fillna_mean', SimpleImputer(strategy='mean'), ['LoanAmount','Loan_Amount_Term']),
     ('fillna_zero', SimpleImputer(strategy='constant',fill_value=0), ['Credit_History'])],
     remainder='passthrough')

In [5]:
log_trans = ColumnTransformer([
     ('log_trans', FunctionTransformer(np.log), ['LoanAmount','total_income'])],
     remainder='passthrough')

In [6]:

preprocessing = Pipeline([
    ('total_income', DataframeFunctionTransformer(create_total_income_feature)),
    ('fillna_trans', fillna_trans),
    ('to_dataframe', DataframeFunctionTransformer(to_dataframe)),
    ('log_trans', log_trans)
])

In [7]:
pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ('encodig', OneHotEncoder(sparse=False, handle_unknown='ignore')),
    ('scaling', MinMaxScaler()),
    ('rf', RandomForestClassifier(n_estimators=1000))
])

In [8]:
from sklearn.model_selection import GridSearchCV
param_grid = {"scaling": [MinMaxScaler(),StandardScaler()],
              'rf__n_estimators': [10,100,1000], 
              'rf__max_depth': [2, 5, None],
              'rf__min_samples_split': [2,10,30]}
# create a Grid Search object
grid_search = GridSearchCV(pipeline, param_grid, n_jobs = -1, verbose=10, refit=True)    
# fit the model and tune parameters
grid_search.fit(X, y)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV 1/5; 1/54] START rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=10, scaling=MinMaxScaler()
[CV 1/5; 1/54] END rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=10, scaling=MinMaxScaler();, score=nan total time=   0.0s
[CV 2/5; 1/54] START rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=10, scaling=MinMaxScaler()
[CV 2/5; 1/54] END rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=10, scaling=MinMaxScaler();, score=nan total time=   0.0s
[CV 3/5; 1/54] START rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=10, scaling=MinMaxScaler()
[CV 3/5; 1/54] END rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=10, scaling=MinMaxScaler();, score=nan total time=   0.0s
[CV 4/5; 1/54] START rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=10, scaling=MinMaxScaler()


<ipython-input-3-8c7c905d6311>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df['total_income'] = input_df['ApplicantIncome'] + input_df['CoapplicantIncome']
/home/sid/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
AttributeError: 'float' object has no attribute 'log'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/sid/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/h

[CV 4/5; 1/54] END rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=10, scaling=MinMaxScaler();, score=nan total time=   0.0s
[CV 5/5; 1/54] START rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=10, scaling=MinMaxScaler()
[CV 5/5; 1/54] END rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=10, scaling=MinMaxScaler();, score=nan total time=   0.0s
[CV 1/5; 2/54] START rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=10, scaling=StandardScaler()
[CV 1/5; 2/54] END rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=10, scaling=StandardScaler();, score=nan total time=   0.0s
[CV 2/5; 2/54] START rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=10, scaling=StandardScaler()
[CV 2/5; 2/54] END rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=10, scaling=StandardScaler();, score=nan total time=   0.0s
[CV 3/5; 2/54] START rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=10, scaling=StandardScaler()
[CV 3/5; 2/54] END

/home/sid/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
AttributeError: 'float' object has no attribute 'log'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/sid/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/sid/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/sid/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/pipeline.py", line 303, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/home/sid/anaconda3/envs/myenv/lib/python3.8/site-packages/joblib/memory.py", line 352, in __call__
 

[CV 5/5; 2/54] END rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=10, scaling=StandardScaler();, score=nan total time=   0.0s
[CV 1/5; 3/54] START rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=100, scaling=MinMaxScaler()
[CV 1/5; 3/54] END rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=100, scaling=MinMaxScaler();, score=nan total time=   0.0s
[CV 2/5; 3/54] START rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=100, scaling=MinMaxScaler()
[CV 2/5; 3/54] END rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=100, scaling=MinMaxScaler();, score=nan total time=   0.0s
[CV 3/5; 3/54] START rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=100, scaling=MinMaxScaler()
[CV 3/5; 3/54] END rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=100, scaling=MinMaxScaler();, score=nan total time=   0.0s
[CV 4/5; 3/54] START rf__max_depth=2, rf__min_samples_split=2, rf__n_estimators=100, scaling=MinMaxScaler()
[CV 4/5; 3/54] END 

...

<ipython-input-3-8c7c905d6311>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df['total_income'] = input_df['ApplicantIncome'] + input_df['CoapplicantIncome']
/home/sid/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
AttributeError: 'float' object has no attribute 'log'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/sid/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/h

[CV 2/5; 53/54] END rf__max_depth=None, rf__min_samples_split=30, rf__n_estimators=1000, scaling=MinMaxScaler();, score=nan total time=   0.0s
[CV 3/5; 53/54] START rf__max_depth=None, rf__min_samples_split=30, rf__n_estimators=1000, scaling=MinMaxScaler()
[CV 3/5; 53/54] END rf__max_depth=None, rf__min_samples_split=30, rf__n_estimators=1000, scaling=MinMaxScaler();, score=nan total time=   0.0s
[CV 4/5; 53/54] START rf__max_depth=None, rf__min_samples_split=30, rf__n_estimators=1000, scaling=MinMaxScaler()
[CV 4/5; 53/54] END rf__max_depth=None, rf__min_samples_split=30, rf__n_estimators=1000, scaling=MinMaxScaler();, score=nan total time=   0.1s
[CV 5/5; 53/54] START rf__max_depth=None, rf__min_samples_split=30, rf__n_estimators=1000, scaling=MinMaxScaler()
[CV 5/5; 53/54] END rf__max_depth=None, rf__min_samples_split=30, rf__n_estimators=1000, scaling=MinMaxScaler();, score=nan total time=   0.0s
[CV 1/5; 54/54] START rf__max_depth=None, rf__min_samples_split=30, rf__n_estimators=1

/home/sid/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
AttributeError: 'float' object has no attribute 'log'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/sid/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/sid/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/sid/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/pipeline.py", line 303, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/home/sid/anaconda3/envs/myenv/lib/python3.8/site-packages/joblib/memory.py", line 352, in __call__
 

[CV 2/5; 54/54] END rf__max_depth=None, rf__min_samples_split=30, rf__n_estimators=1000, scaling=StandardScaler();, score=nan total time=   0.0s
[CV 3/5; 54/54] START rf__max_depth=None, rf__min_samples_split=30, rf__n_estimators=1000, scaling=StandardScaler()
[CV 3/5; 54/54] END rf__max_depth=None, rf__min_samples_split=30, rf__n_estimators=1000, scaling=StandardScaler();, score=nan total time=   0.0s
[CV 4/5; 54/54] START rf__max_depth=None, rf__min_samples_split=30, rf__n_estimators=1000, scaling=StandardScaler()
[CV 4/5; 54/54] END rf__max_depth=None, rf__min_samples_split=30, rf__n_estimators=1000, scaling=StandardScaler();, score=nan total time=   0.0s
[CV 5/5; 54/54] START rf__max_depth=None, rf__min_samples_split=30, rf__n_estimators=1000, scaling=StandardScaler()
[CV 5/5; 54/54] END rf__max_depth=None, rf__min_samples_split=30, rf__n_estimators=1000, scaling=StandardScaler();, score=nan total time=   0.0s


TypeError: loop of ufunc does not support argument 0 of type float which has no callable log method